In [ ]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import make_scorer, f1_score, confusion_matrix, roc_curve, accuracy_score, auc, roc_auc_score, classification_report
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt


In [3]:
data = pd.read_csv('data/processed/mushrooms_pca.csv')
y = data['class']
X = data.drop('class', axis=1)

X_train, X_test, y_train, y_test = \
        train_test_split(X, y.ravel(), test_size=.1, random_state=42)


In [9]:
grid_search = GridSearchCV(clf, tuned_parameters, scoring='f1',
                           cv=10, return_train_score=True, n_jobs=3)
grid_search.fit(X_train, y_train)


In [45]:
res=pd.DataFrame(grid_search.cv_results_).sort_values('rank_test_score').reset_index(drop=True)
res = res[['param_kernel', 'param_coef0', 'param_C', 'mean_test_score', 'mean_fit_time']]
print(res.head(n=8).to_latex())

\begin{tabular}{llllrr}
\toprule
{} & param\_kernel & param\_coef0 & param\_C &  mean\_test\_score &  mean\_fit\_time \\
\midrule
0 &       linear &         NaN &       1 &         0.857536 &       4.923426 \\
1 &       linear &         NaN &    0.01 &         0.853436 &       3.529057 \\
2 &       linear &         NaN &     0.1 &         0.853145 &       3.212004 \\
3 &          rbf &         NaN &       1 &         0.844371 &       5.537084 \\
4 &         poly &         0.5 &     0.1 &         0.834283 &       6.544490 \\
5 &         poly &         0.5 &     0.1 &         0.824538 &       6.869619 \\
6 &          rbf &         NaN &       1 &         0.634898 &       8.441214 \\
7 &          rbf &         NaN &     0.1 &         0.611424 &       8.392245 \\
\bottomrule
\end{tabular}



In [31]:
pickle.dump(grid_search, open("models/svm.p", "wb" ) )
y_true, y_pred = y_test, grid_search.predict(X_test)
grid_search
print(grid_search.best_params_)
report = classification_report(y_true, y_pred)
print(report)

{'C': 1, 'kernel': 'linear'}
              precision    recall  f1-score   support

           0       0.87      0.88      0.88       843
           1       0.87      0.85      0.86       782

   micro avg       0.87      0.87      0.87      1625
   macro avg       0.87      0.87      0.87      1625
weighted avg       0.87      0.87      0.87      1625



In [47]:
confusion_matrix(y_true, y_pred)

array([[773,  70],
       [ 78, 704]], dtype=int64)

In [5]:
pd.DataFrame(grid_search.cv_results_)
print(pd.DataFrame(grid_search.cv_results_).to_latex)

NameError: name 'grid_search' is not defined

In [15]:

fpr, tpr, thresholds = roc_curve(y_test, y_pred, pos_label=1)
roc_auc = auc(fpr, tpr)
plt.figure()

lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

In [4]:
clf = MLPClassifier()

grid_search_nn = GridSearchCV(clf, nn_tuned_parameters, scoring='f1',
                           cv=10, return_train_score=True, n_jobs=3)
grid_search_nn.fit(X_train, y_train)


C:\Users\user\Desktop\kto_ma_szkole_ten_ma_lzej\Erasm\mushrooms\venv\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False),
       fit_params=None, iid='warn', n_jobs=3,
       param_grid=[{'hidden_layer_sizes': [(30,), (50,), (50, 10), (100, 10, 10)], 'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10], 'max_iter': range(50, 200, 50)}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='f1', verbose=0)

In [46]:
y_true, y_pred = y_test, grid_search_nn.predict(X_test)
print(grid_search_nn.best_params_)
report = classification_report(y_true, y_pred)
print(report)

{'alpha': 0.1, 'hidden_layer_sizes': (50,), 'max_iter': 100}
              precision    recall  f1-score   support

           0       0.91      0.92      0.91       843
           1       0.91      0.90      0.90       782

   micro avg       0.91      0.91      0.91      1625
   macro avg       0.91      0.91      0.91      1625
weighted avg       0.91      0.91      0.91      1625



In [49]:
res=pd.DataFrame(grid_search_nn.cv_results_).sort_values('rank_test_score').reset_index(drop=True)
res = res[['param_hidden_layer_sizes', 'param_alpha', 'param_max_iter', 'mean_test_score', 'mean_fit_time']]
print(res.head(n=8).to_latex())
pickle.dump(grid_search_nn, open("models/nn.p", "wb" ) )

\begin{tabular}{llllrr}
\toprule
{} & param\_hidden\_layer\_sizes & param\_alpha & param\_max\_iter &  mean\_test\_score &  mean\_fit\_time \\
\midrule
0 &                    (50,) &         0.1 &            100 &         0.901093 &       5.726179 \\
1 &                 (50, 10) &           1 &             50 &         0.899865 &       3.047146 \\
2 &                    (50,) &         0.1 &             50 &         0.899273 &       2.809682 \\
3 &                    (50,) &        0.01 &             50 &         0.899199 &       3.005558 \\
4 &                 (50, 10) &           1 &            150 &         0.899135 &       8.787189 \\
5 &            (100, 10, 10) &           1 &             50 &         0.898126 &       4.338791 \\
6 &                    (50,) &      0.0001 &             50 &         0.897904 &       3.261373 \\
7 &                    (50,) &         0.1 &            150 &         0.897216 &       8.286927 \\
\bottomrule
\end{tabular}



   param_hidden_layer_sizes param_alpha param_max_iter  mean_test_score  \
0                     (50,)         0.1            100         0.901093   
23                    (30,)         0.1            100         0.893007   
26                 (50, 10)         0.1            100         0.892127   
45            (100, 10, 10)         0.1            100         0.888268   

    mean_fit_time  
0        5.726179  
23       4.458571  
26       6.274711  
45       9.023557  


In [64]:
f1scores = res_alpha['mean_test_score']
plt.plot(res_alpha['param_alpha'], f1scores)
plt.xlabel('alpha')
plt.ylabel('f1 score')
plt.title('MLP classifier performance')

plt.show()